<a href="https://colab.research.google.com/github/IjeomaM/dotnet-project/blob/main/latest_Big_Data_Assessment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("Big Data Assessment")
!ls
!rm -f spark-3.4.[01]-bin-hadoop3.tgz*
!rm -rf spark-3.4.[01]-bin-hadoop3
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz
!tar -xf spark-3.4.3-bin-hadoop3.tgz
!ls -alt

Big Data Assessment
sample_data
--2024-05-02 10:22:00--  https://downloads.apache.org/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.208.237, 135.181.214.104, 2a01:4f8:10a:39da::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.208.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388930980 (371M) [application/x-gzip]
Saving to: ‘spark-3.4.3-bin-hadoop3.tgz’

spark-3.4.3-bin-had 100%[===================>] 370.91M  20.1MB/s    in 20s     

2024-05-02 10:22:20 (19.0 MB/s) - ‘spark-3.4.3-bin-hadoop3.tgz’ saved [388930980/388930980]

total 379836
drwxr-xr-x  1 root root      4096 May  2 10:22 .
drwxr-xr-x  1 root root      4096 May  2 10:16 ..
drwxr-xr-x  1 root root      4096 Apr 30 13:20 sample_data
drwxr-xr-x  4 root root      4096 Apr 30 13:19 .config
-rw-r--r--  1 root root 388930980 Apr 15 01:30 spark-3.4.3-bin-hadoop3.tgz
drwxr-xr-x 13 1000 1000      4096 Apr 15 01:28 spark

In [4]:
# install findspark if not already installed
!pip3 install findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
# the next line gives us 'local' mode. try 'local[2]' to use 2 cores or 'master:NNNN' to run on Spark standalone cluster at port NNNN
spark_conf = SparkConf().setMaster('local[2]').setAppName('MyApp')
sc = SparkContext(conf=spark_conf)
# see what we have by examining the Spark User Interface
from pyspark.sql import *
from pyspark.sql.functions import *

In [6]:
# TO DO:  install findspark
import findspark

In [7]:
# get bike hire file for given year from TfL open data
!wget https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2014.zip
!unzip cyclehireusagestats-2014.zip

--2024-05-02 10:23:37--  https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2014.zip
Resolving cycling.data.tfl.gov.uk (cycling.data.tfl.gov.uk)... 104.16.98.104, 104.16.97.104
Connecting to cycling.data.tfl.gov.uk (cycling.data.tfl.gov.uk)|104.16.98.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225215129 (215M) [application/zip]
Saving to: ‘cyclehireusagestats-2014.zip’

cyclehireusagestats 100%[===================>] 214.78M   105MB/s    in 2.1s    

2024-05-02 10:23:39 (105 MB/s) - ‘cyclehireusagestats-2014.zip’ saved [225215129/225215129]

Archive:  cyclehireusagestats-2014.zip
  inflating: 1. Journey Data Extract 05Jan14-02Feb14.csv  
  inflating: 10a. Journey Data Extract 14Sep14-27Sep14.csv  
  inflating: 10b. Journey Data Extract 28Sep14-11Oct14.csv  
  inflating: 11a. Journey Data Extract 12Oct14-08Nov14.csv  
  inflating: 11b. Journey Data Extract 12Oct14-08Nov14.csv  
  inflating: 12a. Journey Data Extract 09Nov14-06Dec14.csv  
  i

In [8]:
# read in file

!ls
file="./*csv"
spark = SparkSession.builder.appName("bikes").getOrCreate()
Rides_df = (spark.read.format("csv")
         .option("header", "true")
         .option("inferSchema", "true")
         .load(file))

# show top 10
Rides_df.show(10)

'10a. Journey Data Extract 14Sep14-27Sep14.csv'  '4. Journey Data Extract 01Apr14-26Apr14.csv'
'10b. Journey Data Extract 28Sep14-11Oct14.csv'  '5. Journey Data Extract 27Apr14-24May14.csv'
'11a. Journey Data Extract 12Oct14-08Nov14.csv'  '6. Journey Data Extract 25May14-21Jun14.csv'
'11b. Journey Data Extract 12Oct14-08Nov14.csv'  '7. Journey Data Extract 22Jun14-19Jul14.csv'
'12a. Journey Data Extract 09Nov14-06Dec14.csv'  '8a Journey Data Extract 20Jul14-31Jul14.csv'
'12b. Journey Data Extract 09Nov14-06Dec14.csv'  '8b Journey Data Extract 01Aug14-16Aug14.csv'
'13a. Journey Data Extract 07Dec14-21Dec14.csv'  '9a Journey Data Extract 17Aug14-31Aug14.csv'
'13b. Journey Data Extract 22Dec14-03Jan15.csv'  '9b Journey Data Extract 01Sep14-13Sep14.csv'
'14. Journey Data Extract 08Dec13-04Jan14.csv'	  cyclehireusagestats-2014.zip
'1. Journey Data Extract 05Jan14-02Feb14.csv'	  sample_data
'2. Journey Data Extract 03Feb14-01Mar14.csv'	  spark-3.4.3-bin-hadoop3
'3. Journey Data Extract 02Mar

In [9]:
Rides_df.printSchema()

root
 |-- Rental Id: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Bike Id: string (nullable = true)
 |-- End Date: string (nullable = true)
 |-- EndStation Id: string (nullable = true)
 |-- EndStation Name: string (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- StartStation Id: string (nullable = true)
 |-- StartStation Name: string (nullable = true)



In [10]:
# see how many entries (rows) in data
numRows = Rides_df.count()
print("there are ",numRows," rows")

there are  11481596  rows


In [11]:
from pyspark.sql.types import IntegerType, DateType

In [ ]:
# Inspect the DataFrame and cast columns to correct data types
Rides_df = Rides_df.withColumn("Bike Id", col("Bike Id").cast(IntegerType())) \
           .withColumn("Rental Id", col("Rental Id").cast(IntegerType()))\
           .withColumn("Duration", col("Duration").cast(IntegerType()))\
           .withColumn("EndStation Id", col("EndStation Id").cast(IntegerType()))\
           .withColumn("StartStation Id", col("StartStation Id").cast(IntegerType()))

In [ ]:
Rides_df.printSchema()

In [12]:
Ridesnew_df = Rides_df.dropna(subset=["End Date", "Duration","Start Date"])

In [13]:
Ridesnew_df.count()

10242483

In [14]:
# find rows where Duration is zero or less than zero
Ridesnew_df.filter("Duration <= 0").count()

32502

In [15]:
Ridesnew1_df = Ridesnew_df.filter((col("Duration") >0))

In [16]:
Ridesnew1_df.count()

10209981

In [17]:
from pyspark.sql.functions import col, to_date

# Convert the "Start Date" column from string to date format and filter out null values
Ridesnew1_df = Ridesnew1_df.withColumn("Start Date", to_date(col("Start Date"), "dd/MM/yyyy HH:mm")) \
                         .filter(col("Start Date").isNotNull())

Ridesnew1_df = Ridesnew1_df.withColumn("End Date", to_date(col("End Date"), "dd/MM/yyyy HH:mm")) \
                         .filter(col("End Date").isNotNull())

Ridesnew1_df.show(10)
Ridesnew1_df.count()


+---------+--------+-------+----------+-------------+--------------------+----------+---------------+--------------------+
|Rental Id|Duration|Bike Id|  End Date|EndStation Id|     EndStation Name|Start Date|StartStation Id|   StartStation Name|
+---------+--------+-------+----------+-------------+--------------------+----------+---------------+--------------------+
| 34263367|    1080|   9076|2014-06-24|          695|Islington Green, ...|2014-06-24|            311|Foley Street, Fit...|
| 34603487|     660|   6328|2014-07-03|          695|Islington Green, ...|2014-07-03|             22|Northington Stree...|
| 34689078|     120|   2006|2014-07-05|          357|Howland Street, F...|2014-07-05|            311|Foley Street, Fit...|
| 34724273|    1260|   7904|2014-07-06|          695|Islington Green, ...|2014-07-06|            311|Foley Street, Fit...|
| 34956750|    1500|   8251|2014-07-13|           93|Cloudesley Road, ...|2014-07-13|            311|Foley Street, Fit...|
| 34946549|    1

10209981

In [22]:
Ridesnew1_df.printSchema()

root
 |-- Rental Id: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Bike Id: string (nullable = true)
 |-- End Date: date (nullable = true)
 |-- EndStation Id: string (nullable = true)
 |-- EndStation Name: string (nullable = true)
 |-- Start Date: date (nullable = true)
 |-- StartStation Id: string (nullable = true)
 |-- StartStation Name: string (nullable = true)



In [23]:
#Filter and count where Start Date is not 2014
Ridesfiltered_df = Ridesnew1_df.filter((col("Start Date") >= "2015-01-01") | (col("Start Date") <= "2013-12-31"))
Ridesfiltered_df.show(20)
Ridesfiltered_df.count()


+---------+--------+-------+----------+-------------+--------------------+----------+---------------+--------------------+
|Rental Id|Duration|Bike Id|  End Date|EndStation Id|     EndStation Name|Start Date|StartStation Id|   StartStation Name|
+---------+--------+-------+----------+-------------+--------------------+----------+---------------+--------------------+
| 29440493|    1380|   1056|2013-12-12|          695|Islington Green, ...|2013-12-12|            311|Foley Street, Fit...|
| 29458775|    1680|   8123|2013-12-13|          695|Islington Green, ...|2013-12-13|            311|Foley Street, Fit...|
| 29533719|    1200|   6986|2013-12-18|          695|Islington Green, ...|2013-12-18|            311|Foley Street, Fit...|
| 29552829|    1320|   9054|2013-12-19|          695|Islington Green, ...|2013-12-19|            311|Foley Street, Fit...|
| 29660647|     180|   6282|2013-12-28|          695|Islington Green, ...|2013-12-28|            234|Liverpool Road (N...|
| 29575688|     

345690

In [24]:
RidesUpdated_df = Ridesnew1_df.filter(~((col("Start Date") >= "2015-01-01") | (col("Start Date") <= "2013-12-31")))
RidesUpdated_df.count()

9864291

In [25]:
# Count the number of rows before dropping duplicates
total_rows = RidesUpdated_df.count()
#Count the number of rows after dropping duplicates
unique_rows = RidesUpdated_df.dropDuplicates().count()
# Calculate the number of duplicates
num_duplicates = total_rows - unique_rows
print("Number of duplicates in RidesFinal_df:", num_duplicates)

Number of duplicates in RidesFinal_df: 0


In [26]:
# Remove outliners using IQR
#Calculate the first and third quartiles
q1, q3 = RidesUpdated_df.approxQuantile("Duration", [0.25, 0.75], 0.05)

# Calculate the interquartile range (IQR)
iqr = q3 - q1

# Define lower and upper bounds for outlier detection
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filter out rows with values outside the bounds

RidesFinal_df = RidesUpdated_df.filter((col("Duration") >= lower_bound) & (col("Duration") <= upper_bound))

print(f"Dataframe without outliers: {RidesFinal_df.count()}")

IllegalArgumentException: requirement failed: Quantile calculation for column Duration with data type StringType is not supported.

In [ ]:

# import Python libs
import matplotlib.pyplot as plt
import pandas as pd

# look at values (random sampling)
sample_df=RidesFinal_df.sample(0.001) # 0.001 of 0.5M is 500
print("sample has ",sample_df.count()," elements")
plotSample_pdf = sample_df[["Bike Id", "Duration"]].toPandas()
plotSample_pdf.plot(kind="hist", y="Duration", x="Bike Id") # Hist plot (duration .v. bikeID)
plt.show()

In [ ]:
RidesFinal_df.agg({"Duration": "min"}).show()
RidesFinal_df.agg({"Duration": "mean"}).show()
RidesFinal_df.agg({"Duration": "max"}).show()
RidesFinal_df.agg({"Duration": "skewness"}).show()

In [ ]:
# so vast majority of rides in this month have "Duration" under 2 hours
total = RidesFinal_df.count()
numExceed = RidesFinal_df.filter("Duration > 7200").count()
print("Of all rides, percentage over 2 hours:", (numExceed*100)/total)

In [ ]:
# TO DO: add a histogram of ride Duration less or equal to 2 hours

less_equal2_df = RidesFinal_df.filter("Duration <= 7200")

plot_pdf = less_equal2_df[["Duration"]].toPandas() # take just "Duration" col, convert to pandas dataframe structure ready to plot
#print("we have ", plot_df.count()," elements to plot")
print("we use '_pdf' to denote 'pandas data frame")

plot_pdf.plot(kind="hist")
plt.show()

In [ ]:
# can refine further
# e.g. remove rides with Duration of 0 seconds or over 3600
rides_df = RidesFinal_df.filter("Duration > 0").filter("Duration <= 3600")[["Duration"]]
# now convert to Pandas df and plot
rides_df.toPandas().plot(kind="hist")
plt.show()

In [ ]:
# TO DO: manually use threshold for given month
print("histogram with 100 bins and log of frequency for rides lasting more than 5 minutes and less than 10mins")
RidesFinal_df.filter("Duration > 300").filter("Duration < 600")[["Duration"]].toPandas().plot(kind="hist", bins=10, log=True)
plt.show()

In [ ]:
# group "StartStation Id" and sum their "Duration", sort in descending order
StartStation_df = (RidesFinal_df.select("StartStation Id", "StartStation Name", "EndStation Id", "EndStation Name","Duration")
         .groupBy("StartStation Id","StartStation Name")).agg(sum("Duration").alias("Total Duration")).orderBy("Total Duration", ascending=False)
# show top 10
StartStation_df.show(10)

print('Note "Duration" is in seconds (see above table)')

In [ ]:
 # group "EndStation Id" and sum their "Duration", sort in descending order
EndStation_df = RidesFinal_df.select("EndStation Id", "EndStation Name", "Duration")\
.groupBy("EndStation Id", "EndStation Name").agg(sum("Duration").alias("Total Duration")).orderBy("Total Duration", ascending=False)
# show top 10
EndStation_df.show(10)

In [ ]:
#determine %age of rides >3 days and interpret what this means


# Count the number of rides with duration greater than 3 days
Rides_greater_3_days = RidesFinal_df.filter(col("Duration") > (259200))  # Assuming duration is in minutes, convert 3 days to minutes (3 * 24 * 60)

# Calculate the total number of rides
Total_rides = RidesFinal_df.count()
Rides_greater_3_days.count()

# Calculate the percentage of rides greater than 3 days
Percentage_greater_3_days = (Rides_greater_3_days.count() / Total_rides) * 100

print("Percentage of rides greater than 3 days:", Percentage_greater_3_days)

In [ ]:
#determine how many rows have Duration of over a day
# Calculate the number of rows where the duration is over a day
rows_over_day = RidesFinal_df.filter(col("Duration") > (24 * 60))  # Assuming duration is in minutes, convert 1 day to minutes (24 * 60)

# Count the number of rows
count_over_day = rows_over_day.count()

print("Number of rows with duration over a day:", count_over_day)

In [ ]:
#Determine popular routes by 'mode' (e.g. short, day hire, commuting etc)
from pyspark.sql.functions import when

# Define rules for determining the mode based on ride duration
mode_description = (
    when(col("Duration") <= 30, "short")  # Rides with duration less than or equal to 30 minutes are considered "short"
    .when((col("Duration") > 30) & (col("Duration") <= 1440), "Day Hire")  # Rides between 30 minutes and 1 day are considered "day hire"
    .otherwise("Commuting")  # All other rides are considered "commuting"
)

# Create the new "Mode" field based on the defined rules
RidesFinal_Mode_df = RidesFinal_df.withColumn("Mode", mode_description)

# Filter the dataset based on the mode
ShortMode_df = RidesFinal_Mode_df.filter(col("Mode") == "Short")
DayHireMode_df = RidesFinal_Mode_df.filter(col("Mode") == "Day Hire")
CommutingMode_df = RidesFinal_Mode_df.filter(col("Mode") == "Commuting")

RidesFinal_Mode_df.show(10)

# Group and aggregate data of the modes tondetermine the popular routes
ShortPopularRoutes_df = (ShortMode_df
                           .groupBy("StartStation Id", "EndStation Id")
                           .count()
                           .orderBy("count", ascending=False))

DayHirePopularRoutes_df = (DayHireMode_df
                              .groupBy("StartStation Id", "EndStation Id")
                              .count()
                              .orderBy("count", ascending=False))

CommutingPopularRoutes_df = (CommutingMode_df
                                .groupBy("StartStation Id", "EndStation Id")
                                .count()
                                .orderBy("count", ascending=False))

print(f"Popular Routes for Short Duration: {ShortPopularRoutes_df.count()}")
print(f"Popular Routes for a Day Hire Duration: {DayHirePopularRoutes_df.count()}")
print(f"Popular Routes for Communting Duration: {CommutingPopularRoutes_df.count()}")

In [ ]:
# Filter rows where the "station name" column equals "Baylis Road, Waterloo"
Baylis_df = RidesFinal_df.filter(col("StartStation Name") == "Baylis Road, Waterloo")

# Show the resulting DataFrame
Baylis_df.show()


In [ ]:
# Calculate the mean duration of Baylis Station
mean_duration = Baylis_df.agg({"Duration": "mean"}).collect()[0][0]

print("Mean Duration for Baylis Road, Waterloo:", mean_duration)

In [ ]:

# Filter rows where the "station name" column is not "Baylis Road, Waterloo"
other_stations_df = RidesFinal_df.filter(~(col("StartStation Name") == "Baylis Road, Waterloo"))

# Show the resulting DataFrame
other_stations_df.show()

In [ ]:
# Calculate the mean duration of other stations
mean_duration_other_stations = other_stations_df.agg({"Duration": "mean"}).collect()[0][0]

print("Mean Duration for Other Stations:", mean_duration_other_stations)

In [ ]:

from pyspark.sql.functions import month, to_date

# Extract month from the "Start Date" column for both DataFrames
Baylis_month_df = Baylis_df.withColumn("Start Date", to_date("Start Date", "dd-MM-yyyy mm:ss"))\
                     .withColumn("Month",month("Start Date"))
other_stations_month_df = other_stations_df.withColumn("Start Date", to_date("Start Date", "dd-MM-yyyy mm:ss"))\
                     .withColumn("Month",month("Start Date"))

# Show the resulting DataFrames
Baylis_month_df.show()
other_stations_month_df.show()


In [ ]:
#from pyspark.sql.functions import month, col, to_date
# Extract month from the "Start Date" column for both DataFrames
Baylis_df = Baylis_df.withColumn("Month", month("Start Date"))
Other_stations_df = other_stations_df.withColumn("Month", month("Start Date"))

# Group by month and StartStation Name for both DataFrames
Baylis_Monthgrouped_df = Baylis_df.groupBy("Month", "StartStation Name").count()
Other_stations_Monthgrouped_df = Other_stations_df.groupBy("Month", "StartStation Name").count()

# Show the resulting DataFrames
Baylis_Monthgrouped_df.show()
Other_stations_Monthgrouped_df.show()


In [ ]:
from pyspark.sql.functions import mean

# Group by month and calculate the mean duration for Baylis station
Baylis_mean_duration_df = Baylis_df.groupBy("Month").agg(mean("Duration").alias("Baylis Mean Duration"))

# Group by month and calculate the mean duration for other stations
Other_stations_mean_duration_df = Other_stations_df.groupBy("Month").agg(mean("Duration").alias("Other Stations Mean Duration"))

# Show the resulting DataFrames
Baylis_mean_duration_df.show()
Other_stations_mean_duration_df.show()

In [ ]:
# Join the two DataFrames on the "Month" column
Baylis_Others_df = Baylis_mean_duration_df.join(Other_stations_mean_duration_df, "Month", "outer")

# Show the resulting DataFrame
Baylis_Others_df.show()


In [ ]:
import matplotlib.pyplot as plt

# Extract data from the joined DataFrame
months = Baylis_Others_df.select("Month").collect()
baylis_mean_duration = Baylis_Others_df.select("Baylis Mean Duration").collect()
other_stations_mean_duration = Baylis_Others_df.select("Other Stations Mean Duration").collect()

# Convert the data to lists for plotting
months = [row["Month"] for row in months]
baylis_mean_duration = [row["Baylis Mean Duration"] for row in baylis_mean_duration]
other_stations_mean_duration = [row["Other Stations Mean Duration"] for row in other_stations_mean_duration]

# Plot the mean duration for Baylis station
plt.plot(months, baylis_mean_duration, label="Baylis Road, Waterloo")

# Plot the mean duration for other stations
plt.plot(months, other_stations_mean_duration, label="Other Stations")

# Set labels and title
plt.xlabel("Month")
plt.ylabel("Mean Duration")
plt.title("Mean Duration Comparison")
plt.legend()

# Show plot
plt.show()


In [ ]:
from pyspark.sql.functions import count

# Group by EndStation Name and count the frequency
station_frequency = RidesFinal_df.groupBy("EndStation Name").agg(count("*").alias("Frequency")).orderBy("Frequency", ascending=False)

# Show the resulting DataFrame
station_frequency.show()

In [ ]:
# Plot routes from Baylis to other stations(heatmap for popularity)

# Filter rows where the StartStation Name is Baylis
baylis_df = RidesFinal_df.filter(col("StartStation Name") == "Baylis Road, Waterloo")

# Filter the top ten stations
top_ten_stations = station_frequency.limit(20)

# Join the filtered Baylis DataFrame with the station_frequency DataFrame
baylis_with_frequency_df = baylis_df.join(top_ten_stations, "EndStation Name", "inner")

# Aggregate data to count frequency of rides from Baylis to each top station
baylis_to_top_stations_freq = baylis_with_frequency_df.groupBy("StartStation Name", "EndStation Name").count()

# Convert DataFrame to a format suitable for plotting heatmap
heatmap_data = baylis_to_top_stations_freq.toPandas()

# Plot heatmap
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
heatmap = sns.heatmap(heatmap_data.pivot(index="StartStation Name", columns="EndStation Name", values="count"), cmap="YlGnBu", annot=True, fmt="g")
heatmap.set_title("Popularity Heatmap: Baylis Station to Top 10 Stations")
plt.xlabel("End Station")
plt.ylabel("Start Station (Baylis)")
plt.show()


In [ ]:
import pandas as pd
from scipy.stats import ttest_ind

# Convert joined DataFrame to Pandas DataFrame
Baylis_Others_df_pandas = Baylis_Others_df.toPandas()

# Perform two-sample t-test
t_stat, p_value = ttest_ind(Baylis_Others_df_pandas["Baylis Mean Duration"],
                            Baylis_Others_df_pandas["Other Stations Mean Duration"],
                            alternative='two-sided')

# Set significance level
alpha = 0.05



print(f"P_value: {p_value}")

# Interpret results
if p_value < alpha:
    print("Reject Null Hypothesis: There is evidence to suggest that the mean duration of rides from Baylis Road is shorter than other stations.")
else:
    print("Fail to Reject Null Hypothesis: There is no evidence to suggest a significant difference in mean ride durations between Baylis Road and other stations.")